In [1]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# import matplotlib.pyplot as plt
# import cv2
from imgaug import augmenters as iaa
import numpy as np
import pandas as pd
# import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import math
import cv2
import tensorflow as tf
import tensorflow.keras as keras
import sys
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.utils import class_weight, shuffle
import albumentations as album
# from one_cycle_lr_tf2 import OneCycleScheduler
# from model_tf2 import create_model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.data_utils import Sequence
# from my_generator_tf2 import MyGenerator
from tensorflow.keras.losses import binary_crossentropy
import random
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger,Callback,LambdaCallback,TensorBoard)
from tensorflow.keras import backend as K
import tempfile
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, Multiply, Lambda,
                          Concatenate, GlobalAveragePooling2D, Softmax)
from tensorflow.keras import metrics
# import pydicom
import glob
from scipy import ndimage
from skimage import exposure

from tensorflow.keras.utils import to_categorical, plot_model

# from sklearn.metrics import f1_score, recall_score, precision_score

# import tensorflow_addons as tfa

In [2]:
def aug_image_label(image, label):
    # input image: (h,w,c) uint8
    # input label: (h,w,c) uint8
    seed_list = [random.randrange(2**32 - 1) for i in range(10)]

    # augmentation pipeline for images
    aug_images = iaa.Sequential([
        iaa.Fliplr(p=0.5, random_state=seed_list[0]), 
        iaa.Flipud(p=0.5, random_state=seed_list[1]), 
        iaa.WithBrightnessChannels(iaa.Add((-50, 50)), random_state=seed_list[2]), 
        iaa.MultiplySaturation((0.5, 1.5), random_state=seed_list[3]), 
        # iaa.BlendAlpha((0.0, 0.1), iaa.contrast.AllChannelsHistogramEqualization(), random_state=seed_list[4]),
        # iaa.Affine(scale=(0.9, 1.1), rotate=(-15, 15), random_state=seed_list[3], cval=255),
        # iaa.PiecewiseAffine(scale=(0.01, 0.05), random_state=seed_list[4], cval=255), 
    ], random_state=seed_list[-1])

    # augmentation pipeline for segmentation maps - with coarse dropout, but without color aug
    aug_segmaps = iaa.Sequential([
        iaa.Fliplr(p=0.5, random_state=seed_list[0]), 
        iaa.Flipud(p=0.5, random_state=seed_list[1]), 
        # iaa.Affine(scale=(0.9, 1.1), rotate=(-15, 15), random_state=seed_list[2]),
        # iaa.PiecewiseAffine(scale=(0.01, 0.05), random_state=seed_list[4]), 
    ], random_state=seed_list[-1])
    
    image_aug = aug_images(image=image)  # First, augment image.
    segmap_aug = aug_segmaps(image=label)  # Second, augment segmentation map.
    return image_aug, segmap_aug

In [ ]:
def normalize(image):
    image = image / 255  # value[0.0-1.0]
    return image.astype('float32')

# function test
# norm_image = normalize(image)
# print("image min =", norm_image.min())
# print("image max =", norm_image.max())
def resize(image, scale=None, desired_size=(128,128)):
    # Compute resize factors
    if scale:
        factor_0 = scale
        factor_1 = scale
    else:
        factor_0 = 1 / (image.shape[0] / desired_size[1])
        factor_1 = 1 / (image.shape[1] / desired_size[0])
    # Resize
    image = ndimage.zoom(image.astype('float32'), (factor_0, factor_1, 1), order=1, output='float32')
    return image.astype('float16')

In [ ]:
def train_preprocessing(sample_id):
    def py_preproc(sample_id):  # input tensor
        # read
        # image = get_image(sample_id)
        # label = get_label(sample_id)
        image = train_arr[sample_id]
        label = label_arr[sample_id]
        # augmentation
        image, label = aug_image_label(image, label)
        # # bilateralFilter
        # image = cv2.bilateralFilter(image, d=d, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)
        # normalize
        image = normalize(image)
        # resize
        image = resize(image, desired_size=(128,128))
        label = resize(label, desired_size=(128,128))   
        # #RGB2GRAY
        # image = np.mean(image, axis=2)
        # image = np.expand_dims(image, axis=-1)

        return image, label
    
    image, label = tf.py_function(py_preproc, [sample_id], [tf.float16, tf.float16])
    return image, label  # tensor output
def valid_preprocessing(sample_id):
    def py_preproc(sample_id):  # input tensor
        # read
        # image = get_image(sample_id)
        # label = get_label(sample_id)
        image = train_arr[sample_id]
        label = label_arr[sample_id]
        # # bilateralFilter
        # image = cv2.bilateralFilter(image, d=d, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)
        # normalize
        image = normalize(image)
        # resize
        image = resize(image, desired_size=(128,128))
        label = resize(label, desired_size=(128,128)) 
        # #RGB2GRAY
        # image = np.mean(image, axis=2)
        # image = np.expand_dims(image, axis =-1)

        return image, label
    
    image, label = tf.py_function(py_preproc, [sample_id], [tf.float16, tf.float16])
    return image, label  # tensor output